# Machine Learning - Practical 4

Names: Smit Patel, Nishtha Agarwal, Daria Bilich

In [ ]:
import numpy as np
import cvxopt
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.datasets import load_wine
from sklearn.tree import DecisionTreeClassifier

## Task 0: The Data

We will work with the data from Practical 3. Load the data and split it into a training and test set. You can re-use the data splitting function from Practical 2.

In [ ]:
def split_data(X, y, frac=0.3, seed=None):
    if seed is not None:
        np.random.seed(seed)
        
    # ---------------- INSERT CODE ----------------------
    indices = np.random.choice( len(X), int((1-frac)*len(X)), replace=False )
    X_train = X[indices]
    y_train = y[indices]

    indices = list((set(range(len(X))) - set(indices)))
    X_test = X[indices]
    y_test = y[indices]

    # ---------------- END CODE -------------------------
    
    return X_train, X_test, y_train, y_test

In [ ]:
# load data
X_2d, t_2d = np.load('nonlin_2d_data.npy')[:,:2], np.load('nonlin_2d_data.npy')[:, 2]

In [ ]:
# split data
X_train, X_test, t_train, t_test = split_data(X_2d, t_2d, seed=1)

## Task 1: Support Vector Machines

First, you will implement a training algorithm for the Support Vector Machine (SVM). For solving the quadratic program, we provide a simple interface to the cvxopt library below.

In SVMs, each data sample $x_n$ has a corresponding lagrange multiplier $\alpha_n$ which indicates if $x_n$ is a support vector. In the latter case $\alpha_n > 0$ holds. 
The goal of learning the SVM is to figure out which samples are support vectors by learning $\mathbf{\alpha}$. The dual SVM optimizes the following quadratic program.

$$ \min \frac{1}{2} \sum_{n=1}^N \sum_{m=1}^N \alpha_n \alpha_m t_n t_m k(\mathbf{x}_n, \mathbf{x}_m) - \sum_{n=1}^N \alpha_n$$
subject to 
$$ 0 \leq \alpha_n \leq C $$
$$ \sum_{n=1}^N \alpha_n t_n = 0 $$ 

The quadratic program solver expects the following form:
$$ \min \frac{1}{2} \alpha^T P \alpha + \mathbf q^T \mathbf \alpha $$
subject to 

$$A \alpha = b$$
$$G \alpha \leq h $$

Here, $A$ and $G$ are matrices with one row per individual constraint. Similarly, $b$ (the intercept or bias) and $h$ are vectors with one element per individual constraint.

Having trained the SVM, a prediction for an input $\mathbf{x}$ is made by:

$$ y = sign([\sum_n^{N} \alpha_n t_n k(\mathbf{x}, \mathbf{x}_n)] + b)  $$


### Task 1.1
 
Use the code provided below as a basis to express the constrained optimization problem in terms of $P, q, A, b, G$ and $h$ and implement a function `fit_svm` which passes these variables to the provided QP solver. Fit a SVM on the training data and extract its parameters using a linear kernel (dot product).

**Hints:**
  - The box constraint $0 \leq \alpha_n \leq C$ defines two constraints of the form $G \alpha_n \leq h$ for each $\alpha_n$.
  - The inequality $x \geq 0$ is equivalent to $-x \leq 0$.
  - The SVM is described in chapter 12 of Elements of Statistical Learning.

In [31]:
len(cvxopt.matrix(np.vstack((np.identity(len(X_train)) * -1,  np.identity(len(X_train))))))

61250

In [70]:
#testing rbf-smit
np.linalg.norm(X_train[0]-X_train[1])

2.9490544728409547

In [14]:
kernel_mt = np.zeros((len(X_train), len(X_train)))
for i in range(len(X_train)):
  for j in range(len(X_train)):
    kernel_mt[i, j] = linear_kernel(X_train[i], X_train[j])

<175x175 matrix, tc='d'>

In [90]:
def linear_kernel(a, b):
    # ---------------- INSERT CODE ----------------------

    return np.dot(a,b)

    # ---------------- END CODE -------------------------


def fit_svm(X, t, kernel, C=1.0):
  '''Fit SVM using data (X,t), specified kernel and parameter C.
  Inputs
        X:  predictors
        t:  targets
        C:  constant
  '''

  t = np.array([-1 if l == 0 else 1 for l in t])
  kernel_mt = np.zeros((len(X), len(X)))
  for i in range(len(X)):
    for j in range(len(X)):
      kernel_mt[i, j] = kernel(X[i], X[j])
  n_samples = len(X)
  # ---------------- INSERT CODE ----------------------
  P = cvxopt.matrix(np.outer(t, t) * kernel_mt)
  q = cvxopt.matrix(np.ones(len(X)) * -1)
  A = cvxopt.matrix(t, (1, len(X)), tc='d')
  b = cvxopt.matrix(0, tc='d')
  G = cvxopt.matrix(np.vstack((np.identity(n_samples) * -1,  np.identity(n_samples))))
  h = cvxopt.matrix(np.vstack((cvxopt.matrix(np.zeros(n_samples)), cvxopt.matrix(np.ones(n_samples) * C))))
  # ---------------- END CODE -------------------------
  
  assert P.size == (len(X), len(X))
  assert len(q) == len(X)
  assert A.size == (1, n_samples)
  assert b.size == (1, 1)
  #assert len(G) == 2 * len(X)
  #assert len(h) == 2 * len(X)

  return solve_quadratic_program(P, q, A, b, G, h)

def solve_quadratic_program(P, q, A, b, G, h):
    '''Uses cvxopt to solve the quadratic program.'''
    P, q, A, b, G, h = [cvxopt.matrix(var) for var in [P, q, A, b, G, h]]
    minimization = cvxopt.solvers.qp(P, q, G, h, A, b)
    lagr_mult = np.ravel(minimization['x'])
    return lagr_mult


def extract_parameters(X, t, lagr_mult,kernel, threshold=1e-7):
    '''Computes the intercept from the support vector constraints.
    
    Inputs
        X:         predictors
        t:         targets
        kernel:    a kernel to be used
        lagr_mult: the Lagrange multipliers obtained by solving the dual QP
        threshold: threshold for choosing support vectors
    
    Returns
        lagr_mult: lagrange multipliers for the support vectors
        svs:       set of support vectors
        sv_labels: targets t_n for the support vectors
        intercept: computed intercept (also called bias)
    '''
    t = np.array([-1 if l == 0 else 1 for l in t])
    
    # ---------------- INSERT CODE ----------------------
    lagr_mult1 = lagr_mult[lagr_mult > threshold]
    svs = X[lagr_mult > threshold]
    sv_labels = t[lagr_mult > threshold]
    intercept = sv_labels[0]
    for i in range(len(lagr_mult1)):
      intercept -= lagr_mult1[i] * sv_labels[i] * kernel(svs[i], svs[0])
    # ---------------- END CODE -------------------------

    return lagr_mult1, svs, sv_labels, intercept


In [91]:
# Training
# Fit SVM on training data
lagr_mult = fit_svm(X_train, t_train, kernel=linear_kernel, C=1.0)

# Extract parameters
lagr_mul, svs,sv_labels,intercept = extract_parameters(X_train, t_train,lagr_mult=lagr_mult, kernel=linear_kernel)

     pcost       dcost       gap    pres   dres
 0: -1.6611e+02 -3.8981e+02  1e+03  2e+00  3e-14
 1: -1.2559e+02 -2.6081e+02  1e+02  8e-15  2e-14
 2: -1.3748e+02 -1.5048e+02  1e+01  5e-15  1e-14
 3: -1.3799e+02 -1.3813e+02  1e-01  2e-15  1e-14
 4: -1.3800e+02 -1.3800e+02  1e-03  4e-15  1e-14
 5: -1.3800e+02 -1.3800e+02  1e-05  4e-15  2e-14
Optimal solution found.


### Task 1.2

Having learnt an SVM, we can use the calculated parameters to make predictions on novel samples.
- Implement a function `svm_predict(X, kernel, lagr_mult, svs, sv_labels, intercept)`.
- Use this function with the linear kernel and compute the test accuracy on the 2d dataset.
- Visualize the samples form the test set in a scatter plot colored by your predictions

In [92]:
def svm_predict(X, lagr_mult, svs, sv_labels, intercept, kernel):
    ''' Given the learned parameters of the SVM, make a prediction on the test set.
    Inputs
        X:         predictors
        kernel:    a kernel to be used
        lagr_mult: the Lagrange multipliers obtained by solving the dual QP
        svs:       set of support vectors
        sv_labels: targets t_n for the support vectors
        intercept: computed intercept (also called bias)
    
    Returns
        prediction: predictions on novel samples
    '''  
    
    # ---------------- INSERT CODE ----------------------
    n_samples = len(X)
    prediction = []
    for sample in X:
      pred = 0
      for i in range(len(lagr_mult)):
        pred += lagr_mult[i] * sv_labels[i] * kernel(svs[i], sample)
      pred += intercept
      prediction.append(np.sign(pred))

    # ---------------- END CODE -------------------------
    
    return prediction

In [93]:
# Testing
# make predictions for test set
y_pred = svm_predict(X_test,lagr_mul, svs, sv_labels,intercept, kernel=linear_kernel)

In [94]:
# visualize predictions
y_pred, t_test

([-1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0],
 array([1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0.,
        1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1.]))

### Task 1.3

- Instead of using the linear kernel, use the Gaussian RBF kernel defined in Practical 3.
- Compare results on with both kernels with sklearn implementation (SVC)
- Visualize the predictions on the test set, the learned support vectors and the decision boundary for both kernels (Hint: Adapt the decision boundary plot from Practical 3).

In [95]:
def rbf_kernel(a, b):
    # ---------------- INSERT CODE ----------------------
    return np.exp(np.power(np.linalg.norm(a- b),2)*-1)
    # ---------------- END CODE -------------------------

In [97]:
# Fit SVM with rbf kernel and calculate the test accuracy
lagr_mult = fit_svm(X_train, t_train, kernel=rbf_kernel, C=1.0)
lagr_mul, svs,sv_labels,intercept = extract_parameters(X_train, t_train,lagr_mult=lagr_mult, kernel=rbf_kernel)
y_pred = svm_predict(X_test,lagr_mul, svs, sv_labels,intercept, kernel=rbf_kernel)

     pcost       dcost       gap    pres   dres
 0: -7.7018e+00 -2.6272e+02  1e+03  2e+00  5e-16
 1: -2.6542e+00 -1.1766e+02  1e+02  2e-15  6e-16
 2: -8.3809e+00 -2.1823e+01  1e+01  1e-15  6e-16
 3: -9.9541e+00 -1.4530e+01  5e+00  3e-16  3e-16
 4: -1.0601e+01 -1.1741e+01  1e+00  3e-16  3e-16
 5: -1.0840e+01 -1.1127e+01  3e-01  1e-15  3e-16
 6: -1.0903e+01 -1.0986e+01  8e-02  2e-16  3e-16
 7: -1.0927e+01 -1.0941e+01  1e-02  6e-16  3e-16
 8: -1.0932e+01 -1.0933e+01  2e-03  2e-16  3e-16
 9: -1.0932e+01 -1.0932e+01  3e-05  6e-16  3e-16
10: -1.0932e+01 -1.0932e+01  3e-07  9e-16  4e-16
Optimal solution found.


In [99]:
# Fit SVM using sklearn and calculate the test accuracy
y_pred,t_test


([1.0,
  -1.0,
  1.0,
  -1.0,
  1.0,
  1.0,
  1.0,
  -1.0,
  -1.0,
  -1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  -1.0,
  1.0,
  1.0,
  1.0,
  -1.0,
  1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  1.0,
  1.0,
  1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  1.0,
  -1.0,
  -1.0,
  1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  1.0,
  1.0,
  -1.0,
  1.0,
  -1.0,
  -1.0,
  1.0,
  1.0,
  -1.0,
  1.0,
  1.0,
  -1.0,
  -1.0,
  1.0,
  1.0,
  1.0,
  -1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  -1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0],
 array([1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0.,
        1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1.]))

In [ ]:
# Visualize


# Task 2: Decision Trees

Next, we will implement a simple decision tree classifier using the Wine dataset, one of the standard sklearn datasets. 

We will use the Gini impurity as a criterion for splitting. It is defined for a set of labels as
$$ G = \sum_{i=0}^C p(i) * (1- p(i)) $$

Given labels $l$ and split $l_a$ and $l_b$, the weighted removed impurity can be computed by $G(l) - \frac{|l_a|}{|l|}G(l_a) - \frac{|l_b|}{|l|}G(l_b)$.

Here is a simple explanation of the Gini impurity that you may find useful: https://victorzhou.com/blog/gini-impurity/


### Task 2.1

1. Plot the distribution of the first feature of for each class of the wine dataset.
2. Implement a function `gini_impurity(t)` that computes the Gini impurity for an array of labels `t`.
3. Calculate the removed Gini impurity for a split after 50 samples, i.e. between `t[:50]` and `t[50:]`.

In [ ]:
# Load Wine dataset and split into train+test set

X, t = load_wine(return_X_y=True)
X_train, X_test, t_train, t_test = split_data(X, t)

In [ ]:
# Plot distribution


In [ ]:
# Compute Gini impurity


### Task 2.2
For each of the first 12 features, compute the remove Gini impurity for every possible split. Visualize the removed Gini impurity per feature across all splits. Which is the optimal split?

In [ ]:
# Plotting


### Task 2.3

1. Implement a function `build_tree(X, t, depth)` which recursively builds a tree. Use the classes `Node` and `Leaf` as a data structure to build your tree.
2. Implement a function `predict_tree(tree, x)` which makes a prediction for sample `x`. Obtain scores for the `wine` dataset and compare to `sklearn.tree.DecisionTree`.
3. Switch back to the synthetic 2d dataset from the beginning (kernel methods). Compute scores and visualize the decisions in a 2d grid.

In [ ]:
class Node:
    def __init__(self, left, right, n_feat, threshold):
        self.left = left
        self.right = right
        self.n_feat = n_feat
        self.threshold = threshold


class Leaf:
    def __init__(self, label):
        self.label = label


In [ ]:
# Implement recursive tree function

def build_tree(X, t, depth, max_depth=3, n_labels=2):
    
    # ---------------- INSERT CODE ----------------------



    # ---------------- END CODE -------------------------

    
def predict_tree(node, x):
    
    # ---------------- INSERT CODE ----------------------



    # ---------------- END CODE -------------------------

In [ ]:
# Build tree

tree = build_tree(X_train, t_train, 0, max_depth=3, n_labels=3)

In [ ]:
# Calculate training and test scores


In [ ]:
# Calculate test score using sklearn


In [ ]:
# Calculate test score for synthetic 2D dataset


In [ ]:
# Visualize
